In [1]:
import os
import time
import fileinput
import numpy as np  
import matplotlib.pyplot as plt    
from scipy.special import factorial

import pykat.optics.maps as pkm
from pykat import finesse                 
from pykat.commands import *               
from pykat.optics.maps import *            
from IPython.display import display, HTML


%config InlineBackend.figure_format='svg'
%matplotlib inline
pykat.init_pykat_plotting(dpi=90)

                                              ..-
    PyKat 1.2.1           _                  '(
                          \`.|\.__...-""""-_." )
       ..+-----.._        /  ' `            .-'
   . '            `:      7/* _/._\    \   (
  (        '::;;+;;:      `-"' =" /,`"" `) /
  L.        \`:::a:f            c_/     n_'
  ..`--...___`.  .    ,
   `^-....____:   +.      www.gwoptics.org/pykat



## Content:
This notebook is to write a function to generate Zernike maps with the order and amplitude as input

### Note:
Need to set the four corners to zero because Zernike maps only work on unit disk 

In [4]:
def insert(originalfile,string): # This function is to add the map header before the map data 
    with open(originalfile,'r') as f:
        with open('temp.txt','w') as f2: 
            f2.write(string)
            f2.write(f.read())
    os.rename('temp.txt',originalfile)


def Znm(order, n, m, verbose=True, shape=1131, radius=0.15, step_size=0.0002669951063580811): # radius is the cropping radius
    center = (shape-1)/2 # Center of the map
    rrange = radius/step_size # Range of the map such that the radius is 0.15 for a given step_size
    
    def theta(x,y):
        phi = np.arctan2(y, x)
        return phi

    def radial(x,y,n,m):
        if m<0:
            m=-m
        sum=0
        for k in range(int((n-m)/2)+1):   
            r=(-1)**k*factorial(n-k)/factorial(k)/factorial((n+m)/2-k)/factorial((n-m)/2-k)*((x**2+y**2)/(rrange**2))**(n/2-k)
            # Here I used '(x**2+y**2)/(rrange**2)' instead of just '(x**2+y**2)' to normalize the radius
            sum+=r
        return sum

    def angular(x,y,n,m): 
        a=theta(x,y)
        if m>=0:
            angular=np.cos(m*a)
        else:
            angular=-np.sin(m*a)
        return angular

    stepRange = np.arange(shape)-center
    x,y=np.meshgrid(stepRange,stepRange,sparse=True)
    zfunc=radial(x,y,n,m)*angular(x,y,n,m)
    for i in range(shape):
        for j in range(shape): 
            if (i-center)**2+(j-center)**2>= rrange**2:
                zfunc[i][j]=0 # Set the values outside the cropping radius to zero
    mapData=order*zfunc/np.abs(zfunc).max() # Such that the amplitude(maximum value in the map data) equals to 'order'

    
    header =f'''% Surface map
% Name: Z{n}{m}order{order}phasemap
% Type: phase both
% Size: {shape} {shape}
% Optical center (x,y): {center} {center}
% Step size (x,y): {step_size} {step_size}
% Scaling: 1e-09\n\n\n'''
    
    if not (n-m)%2 and step_size*(shape-1)>radius:# Check for right zernike index and right cropping radius, which 
        # has to be smaller than the mirror size
        repo='Znmmaps/'
        filename=f'Z{n}{m}order{order}phasemap.txt'
        np.savetxt(repo+filename, mapData, delimiter=' ',fmt='%1.6f')
        insert(repo+filename, header) 
        if verbose: # If verbose set to false(default is true), it will not plot the map
            fig1=pkm.read_map(repo+filename, mapFormat='finesse').plot()
    else:
        print("Invalid indexes or cropping radius")

In [4]:
for n in range(10):
    for m in range(-n, n+1, 2):
        Znm(1,n,m,shape=1131,radius=0.15,step_size=0.0002669951063580811,verbose=False)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in true_divide
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  


In [5]:
Znm(0,2,2,shape=1131,radius=0.15,step_size=0.0002669951063580811,verbose=False)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in true_divide
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  
